In [1]:
import mysql.connector
import sqlalchemy

db = mysql.connector.connect(
    host=os.environ.get('DB_IP_ADDRESS'),
    user="hussen",
    passwd=os.environ.get('DB_PASSWORD_REMOTE'),
    database="testdatabase"
)

mycursor = db.cursor()



engine = sqlalchemy.create_engine(f"mysql+mysqlconnector://hussen:{os.environ.get('DB_PASSWORD_REMOTE')}@{os.environ.get('DB_IP_ADDRESS')}:3306/testdatabase")



# mycursor.execute('CREATE DATABASE IF NOT EXISTS testdatabase')


In [2]:
import os
import pandas as pd

from googleapiclient.discovery import build

api_key = os.environ.get("YT_API_KEY")

#@lru_cache
def get_comments_and_replies(videoId):

    youtube = build('youtube', 'v3', developerKey=api_key)

    request = youtube.commentThreads().list(
            part = 'snippet',
            videoId = videoId,
            order = 'relevance'
        )

    response = request.execute()

    comments = []

    while response:
        for item in response['items']:
            comment = item['snippet']['topLevelComment']['snippet']['textDisplay']

            comments.append(comment)

        if 'nextPageToken' in response:
            request = youtube.commentThreads().list(
                    part = 'snippet',
                    videoId = videoId,
                    order = 'relevance',
                    pageToken = response['nextPageToken']
                )

            response = request.execute()            

        else:
            response = False


    return comments


In [3]:
df = pd.DataFrame(columns=['videoId', 'comments'])
df['comments'] = get_comments_and_replies(videoId='TT95ZWbpa-I')
df.loc[:,'videoId'] = 'TT95ZWbpa-I'

In [4]:
df.head()

,videoId,comments
0,TT95ZWbpa-I,Get 68% off NordVPN! Only $3.71/mo plus 1 mont...
1,TT95ZWbpa-I,impressive how she so thoughtfully embraces be...
2,TT95ZWbpa-I,Am I the only one who wants a whole movie with...
3,TT95ZWbpa-I,When your roommate studies abroad for a semest...
4,TT95ZWbpa-I,People say vegans are annoying but I’d say peo...


In [5]:
df.to_sql(
    name='comments',
    con=engine,
    if_exists='replace',
    index=False
)

1929

In [6]:
df2 = pd.read_sql_table('comments', con=engine)
df2.head()

,videoId,comments
0,TT95ZWbpa-I,Get 68% off NordVPN! Only $3.71/mo plus 1 mont...
1,TT95ZWbpa-I,impressive how she so thoughtfully embraces be...
2,TT95ZWbpa-I,Am I the only one who wants a whole movie with...
3,TT95ZWbpa-I,When your roommate studies abroad for a semest...
4,TT95ZWbpa-I,People say vegans are annoying but I’d say peo...


In [9]:
mycursor.execute("SHOW TABLES")

print(mycursor.fetchone())


('comments',)
